In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [24]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import Model

The data for this exercise is available at: https://www.kaggle.com/datamunge/sign-language-mnist/home

Sign up and download to find 2 CSV files: sign_mnist_test.csv and sign_mnist_train.csv -- You will upload both of them using this button before you can continue.


In [3]:
uploaded=files.upload()

Saving sign_mnist_test.csv to sign_mnist_test.csv
Saving sign_mnist_train.csv to sign_mnist_train.csv


In [90]:
def get_data(filename):
  # You will need to write code that will read the file passed
  # into this function. The first line contains the column headers
  # so you should ignore it
  # Each successive line contians 785 comma separated values between 0 and 255
  # The first value is the label
  # The rest are the pixel values for that picture
  # The function will return 2 np.array types. One with all the labels
  # One with all the images
  #
  # Tips: 
  # If you read a full line (as 'row') then row[0] has the label
  # and row[1:785] has the 784 pixel values
  # Take a look at np.array_split to turn the 784 pixels into 28x28
  # You are reading in strings, but need the values to be floats
  # Check out np.array().astype for a conversion
    with open(filename) as training_file:
      # Your code starts here
      # Your code ends here
      df = pd.read_csv(filename)
      labels = df.label
      del df['label']
      from sklearn.preprocessing import LabelBinarizer
      label_binarizer = LabelBinarizer()
      labels = label_binarizer.fit_transform(labels)
      images = df.values
      #Using binarizer is more concise, no need to reshape
      #from sklearn.preprocessing import OneHotEncoder
      #labels = np.array(labels).reshape(-1,1)
      #label_binarizer = OneHotEncoder()
      #labels = label_binarizer.fit_transform(labels)
      # Your code ends here
    return images, labels

training_images, training_labels = get_data('sign_mnist_train.csv')
testing_images, testing_labels = get_data('sign_mnist_test.csv')

# Keep these
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

# Their output should be:
# (27455, 28, 28)
# (27455,)
# (7172, 28, 28)
# (7172,)

(27455, 784)
(27455, 24)
(7172, 784)
(7172, 24)


In [91]:
# In this section you will have to add another dimension to the data
# So, for example, if your array is (10000, 28, 28)
# You will need to make it (10000, 28, 28, 1)
# Hint: np.expand_dims
training_images = training_images.reshape(-1,28,28,1)# Your Code Here
testing_images = testing_images.reshape(-1,28,28,1)# Your Code Here
#training_images = np.repeat(training_images,3,axis=3)
#testing_images = np.repeat(testing_images,3,axis=3)
#training_images = tf.image.resize_with_pad(training_images,32,32)
#testing_images = tf.image.resize_with_pad(testing_images,32,32)
# Create an ImageDataGenerator and do Image Augmentation
train_datagen = ImageDataGenerator(
    # Your Code Here
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
    )
train_datagen.fit(training_images)

validation_datagen = ImageDataGenerator(
    # Your Code Here
    rescale = 1./255)

validation_datagen.fit(testing_images)

train_generator = train_datagen.flow(
	training_images,training_labels,  batch_size=128
)

validation_generator = validation_datagen.flow(
	testing_images,testing_labels,
  batch_size=128
)
    
# Keep These
print(training_images.shape)
print(testing_images.shape)
    
# Their output should be:
# (27455, 28, 28, 1)
# (7172, 28, 28, 1)

(27455, 28, 28, 1)
(7172, 28, 28, 1)


In [92]:
# Define the model
# Use no more than 2 Conv2D and 2 MaxPooling2D
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model,Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras import backend as K

# 构建不带分类器的预训练模型
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 24 , activation = 'softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

# 我们继续训练模型，这次我们训练最后两个 Inception block
# 和两个全连接层

    
# The output from model.evaluate should be close to:
[6.92426086682151, 0.56609035]

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_285 (Conv2D)          (None, 28, 28, 75)        750       
_________________________________________________________________
batch_normalization_285 (Bat (None, 28, 28, 75)        300       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 14, 14, 75)        0         
_________________________________________________________________
conv2d_286 (Conv2D)          (None, 14, 14, 50)        33800     
_________________________________________________________________
dropout_39 (Dropout)         (None, 14, 14, 50)        0         
_________________________________________________________________
batch_normalization_286 (Bat (None, 14, 14, 50)        200       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 7, 7, 50)         

[6.92426086682151, 0.56609035]

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)
history = model.fit(train_generator ,epochs = 20 , validation_data = validation_generator , callbacks = [learning_rate_reduction])


model.evaluate(testing_images, testing_labels)

Epoch 1/20
215/215 [==============================] - 111s 515ms/step - loss: 2.8509 - accuracy: 0.1664 - val_loss: 5.7932 - val_accuracy: 0.1145
Epoch 2/20
215/215 [==============================] - 106s 495ms/step - loss: 1.6830 - accuracy: 0.4489 - val_loss: 5.5725 - val_accuracy: 0.0825
Epoch 3/20
215/215 [==============================] - 107s 498ms/step - loss: 1.1015 - accuracy: 0.6278 - val_loss: 1.7099 - val_accuracy: 0.4972
Epoch 4/20
215/215 [==============================] - 107s 497ms/step - loss: 0.8302 - accuracy: 0.7179 - val_loss: 0.4908 - val_accuracy: 0.8134
Epoch 5/20
215/215 [==============================] - 107s 499ms/step - loss: 0.6844 - accuracy: 0.7617 - val_loss: 0.4182 - val_accuracy: 0.8529
Epoch 6/20
215/215 [==============================] - 107s 496ms/step - loss: 0.5723 - accuracy: 0.8046 - val_loss: 1.3156 - val_accuracy: 0.6318
Epoch 7/20
215/215 [==============================] - 107s 496ms/step - loss: 0.4831 - accuracy: 0.8329 - val_loss: 0.2462 -

[701.894775390625, 0.2668711543083191]

In [ ]:
# Plot the chart for accuracy and loss on both training and validation

import matplotlib.pyplot as plt
acc = # Your Code Here
val_acc = # Your Code Here
loss = # Your Code Here
val_loss = # Your Code Here

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()